### Time-Dependent One-Dimensional (1-D) Schrödinger Equation 
###### $ i \hbar \dfrac{\mathrm{d} \psi}{\mathrm{d} t} = H\psi = -\dfrac{\hbar^2}{2m} \, \dfrac{\mathrm{d}^2 \psi}{\mathrm{d} x^2} + V(x)\psi$

In [10]:
import numpy as np
import numpy.linalg as lin
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
matplotlib.rcParams['animation.embed_limit'] = 50

def write_pot(ngridx, pot_grid):
    """
    Save potential as txt file.

    input:
        int         ngridx          Number of x grid points
        np.array    pot_grid        Potential in grid
    """
    strline = ""
    if "float" in str(pot_grid.dtype):
        tmp_grd = pot_grid
    if "complex" in str(pot_grid.dtype):
        tmp_grd = pot_grid.real**2 + pot_grid.imag**2
    for i in range(ngridx):
        strline += "  %.5e " % tmp_grd[i]
    strline += "  %.6e  \n" % tmp_grd[0]
    with open("Potential.txt", 'w') as f:
        f.write(strline)


def write_wave(wave_, E, phi, boxL, ngridt, dt, ngridx, lpacket, ncombistates, au2fs_time):
    """
        Write the probability of the wave function Ψ(t) on a spatial grid for each time step to "wave.txt".

        input:
            np.array    wave_           Constructed Gaussian wave packet
            np.array    E               Eigenvalue vector
            np.array    phi             Eigenvector array
            int         boxL            Box length
            int         ngridt          Number of time steps
            float       dt              Time interval
            int         ngridx          Number of spatial grid points
            int         lpacket         0: use all basis, 1: use N states
            int         ncombistates    Number of eigenstates for linear combination (if lpacket==1)
            float       au2fs_time      Time unit conversion factor
    """
    c_n = wave_.dot(np.conjugate(phi))
    time = np.linspace(0, (ngridt-1)*dt, ngridt)
    position = np.linspace(0, boxL, ngridx)

    # phi_used = phi
    # E_used = E
    phi_used = phi[:, :ncombistates]
    E_used = E[:ncombistates]
    c_n = c_n[:ncombistates]

    if(lpacket == 0):
        Psi_t = (phi_used @ (c_n[:, np.newaxis] * np.exp(-1j * E_used[:, np.newaxis] * time))).T
    if(lpacket == 1):
        bounce = (phi_used @ ( np.exp(-1j * E_used[:, np.newaxis] * time))).T
        a=np.sqrt(np.diag(bounce.dot(np.conjugate(bounce).T)))*dx
        Psi_t=(bounce.T/a).T
        norm_factor = np.sqrt(np.diag(Psi_t.dot(np.conjugate(Psi_t).T)))  # 전체 확률 계산
        Psi_t = (Psi_t.T/norm_factor).T  # 정규화 적용

    if(lpacket == 2):
        from scipy.special import factorial
        Psi_t = ((phi_used * (1/np.sqrt(factorial(list(range(1, ncombistates+1)))))) @ ( np.exp(-1j * E_used[:, np.newaxis] * time))).T

    # 벡터화된 Psi_t 계산 (한 번에 계산)
    # Psi_t = (phi_used @ (c_n[:, np.newaxis] * np.exp(-1j * E_used[:, np.newaxis] * time))).T
    prob = np.abs(Psi_t)**2

    with open("wave.txt", 'w') as f:
        f.write("# t(fs) ")
        f.write(" ".join([f"{pos:.6f}" for pos in position]) + '\n')
        for i, t in enumerate(time * au2fs_time):
            f.write(f"{t:.6f} " + " ".join(f"{p:.8f}" for p in prob[i, :]) + '\n')


def make_wave(boxL, ngridx, pot_shape, sigma, k):
    """
    Construct a Gaussian wave packet and return it on a spatial grid.

    input:
        int         boxL        Box size
        int         ngridx      Number of spatial grid points
        int         pot_shape   Shape of the potential (determines wave packet position)
        float       sigma       Gaussian dispersion
        float       k           Wave vector (momentum)
    output:
        np.array    gridwave    Gaussian wave packet
    """
    gridwave = np.zeros(ngridx, np.complex64)
    dx = np.float64(boxL / ngridx)
    gridwave[:] = 0.0 + 0.j
    midwave = 0.4
    minx = 3
    maxx = 5
    for i in range(ngridx):
        if ((ngridx * minx) // 10 < i and i < (ngridx * maxx) // 10):
            gridwave[i] = np.exp(-((i*dx - midwave*ngridx*dx)**2) / sigma)
    gridwave /= lin.norm(gridwave)
    for i in range(ngridx):
        gridwave[i] = gridwave[i] * np.exp(1j * k * (i*dx - midwave*ngridx*dx))
    return gridwave

def potential(ngridx, pot_shape, pot_height_eV, barrier_thickness):
    """
    Set the shape of potential V and return the potential on a grid.

    input:
        int         ngridx              Number of spatial grid points
        int         pot_shape           Shape of potential:
                                         0 = No potential (Box)
                                         1 = Step potential
                                         2 = Single wall
                                         3 = Double wall
                                         4 = Finite well (packet starts at middle)
                                         5 = Harmonic well
        int         pot_height_eV       Height of the potential barrier (eV)
        int         barrier_thickness    Thickness of the potential barrier (Angstrom)
    output:
        np.array    pot_grid            Potential on the grid
    """
    pot_grid = np.zeros(ngridx)
    pot_grid[0] = 100000000
    pot_grid[ngridx-1] = 100000000

    # Step
    if pot_shape == 1:
        pot_grid[(ngridx//2):(ngridx-2)] = pot_height_eV / har2ev
    # Single wall
    if pot_shape == 2:
        pot_grid[(5*ngridx)//10 : (5*ngridx)//10 + barrier_thickness*10] = pot_height_eV / har2ev
    # Double wall
    if pot_shape == 3:
        pot_grid[(6*ngridx)//10 - barrier_thickness*10 : (6*ngridx)//10] = pot_height_eV / har2ev
        pot_grid[(5*ngridx)//10 : (5*ngridx)//10 + barrier_thickness*10] = pot_height_eV / har2ev
        
    # Square well
    if pot_shape == 4:
        pot_grid[2 : int(ngridx*3.5)//10] = pot_height_eV / har2ev
        pot_grid[int(ngridx*5)//10 : ngridx-2] = pot_height_eV / har2ev
    # Harmonic well
    if pot_shape == 5:
        for i in range(1, ngridx-1):
            pot_grid[i] = ((i - ngridx//2)**2 / (ngridx//2 - 1)**2) * pot_height_eV / har2ev
    return pot_grid

def laplacianCoeff(laplacianDim: int):
    """
    Define FDM points and coefficients for the Laplacian using the Finite Difference Method.

    input:
        int         laplacianDim    Accuracy (dimension) of the Laplacian operator
    output:
        np.array    lcoeff          Coefficients
    """
    a = np.zeros((2*laplacianDim+1, 2*laplacianDim+1))
    c = np.zeros(2*laplacianDim+1)
    c[2] = math.factorial(2)
    for i in range(2*laplacianDim+1):
        for j in range(2*laplacianDim+1):
            a[i, j] = (j - laplacianDim)**i
    inva = lin.inv(a)
    b = np.matmul(inva, c)
    lcoeff = b[laplacianDim:2*laplacianDim+1]
    return lcoeff

def laplacian(ngridx, laplacianDim, dx):
    """
    Define the Laplacian operator matrix for the Hamiltonian.

    input:
        int         ngridx          Number of spatial grid points
        int         laplacianDim    Accuracy (dimension) of the Laplacian operator
        float       dx              Grid spacing
    output:
        np.array    loprt           Laplacian operator matrix
    """
    lcoeff = laplacianCoeff(laplacianDim)
    loprt = np.zeros((ngridx, ngridx))
    for i in range(ngridx):
        for j in range(-laplacianDim, laplacianDim + 1):
            k = i + j
            if k >= ngridx:
                k -= ngridx
            elif k < 0:
                k += ngridx
            loprt[i][k] = lcoeff[abs(j)] / (dx**2)
    return loprt

def hamiltonian(boxL, ngridx, laplacianDim, dx, pot_shape, pot_height_eV, barrier_thickness):
    """
    Define the Hamiltonian using the Laplacian and Potential operators.

    input:
        int         ngridx            Number of spatial grid points
        int         laplacianDim      Accuracy (dimension) of the Laplacian operator
        float       dx                Grid spacing
        int         pot_height_eV
        int         pot_shape         Shape of potential
        int         barrier_thickness Thickness of barrier (Angstrom)
    output:
        np.array    hamiltonian
    """
    pot_op = np.zeros((ngridx, ngridx))
    pot_1d = potential(ngridx, pot_shape, pot_height_eV, barrier_thickness)
    laplacian_op = laplacian(ngridx, laplacianDim, dx)
    np.fill_diagonal(pot_op, val=pot_1d)
    ham_op = -laplacian_op / 2.0 + pot_op
    return ham_op

def read_and_plot(boxL, ngridx, ngridt, ewave, lpacket, ncombistates, interval):
    wave = []
    with open('wave.txt', 'r') as f:
        for line in f.readlines()[1:]:
            wave.append(np.array(line.strip().split(), dtype=float))
    wave = np.array(wave)

    with open('Potential.txt', 'r') as f:
        pot = np.array(f.readline().split(), float)

    x = np.linspace(0, boxL, ngridx - 1)

    if lpacket == 1:
        ewave_scaler = ewave / np.sqrt(ncombistates)
    else:
        ewave_scaler = ewave / har2ev

    visibility_factor = 30

    fig, ax = plt.subplots(figsize=(8, 6))
    line_wave, = ax.plot([], [], label='Wave Packet')
    line_pot, = ax.plot(x, pot[:ngridx - 1], label='Potential', linestyle='-')

    # ax.set_xlim(boxL*0.3, boxL*0.7)
    ax.set_ylim(0, 0.2)
    # ax.set_xticks([])
    # ax.set_yticks([])
    plt.xlabel('Position')
    plt.ylabel('Energy')
    title = ax.set_title('')
    ax.legend()

    def init():
        line_wave.set_data([], [])
        return line_wave, line_pot, title

    def update(frame):
        line_wave.set_data(x, wave[frame + 1, 1:ngridx])
        title.set_text(f'Time = {wave[frame + 1, 0]:.2f} fs')
        return line_wave, title

    ani = animation.FuncAnimation(fig, update, frames=ngridt - 1, 
                                  init_func=init, blit=True, interval=interval*1000)

    plt.close(fig)  # Prevent duplicate display
    display(HTML(ani.to_jshtml()))



def time_dep_schr_eq(pot_shape, pot_height_eV, dx):
    """
    Solve the time-dependent Schrödinger equation using the Hamiltonian eigenvalue problem.
    
    input:
        int         pot_height_eV   Height of the potential wall (eV)
        int         pot_shape       Shape of potential (0: None, 1: Step, 2: Single, 3: Double, 4: Square, 5: Harmonic)
        float       dx              Grid spacing
    output:
        np.array    eigval          Eigenvalues
        np.array    eigvec          Eigenvectors
    """
    ham_op = hamiltonian(boxL_bohr, ngridx, laplacianDim, dx, pot_shape, pot_height_eV, barrier_thickness)
    (eigval, eigvec) = np.linalg.eigh(ham_op)

    
    return eigval, eigvec

if __name__ == "__main__":
    # Simulation parameters
    boxL = 50           # Box Length (Angstrom)
    ngridx = 2001        # Number of grid points

    # Unit conversion factors
    ang2bohr = 1.88973
    har2ev = 27.21140795

    boxL_bohr = boxL * ang2bohr
    dx = np.float64(boxL_bohr / ngridx)
    barrier_thickness = 3  # Barrier thickness (Angstrom)
    au2fs_time = 2.418884326509E-2  # Time conversion (a.u. to fs)
    dt = 0.01 / au2fs_time         # Time grid interval (a.u.)
    laplacianDim = 3

    from ipywidgets import interact_manual
    import ipywidgets as widgets

    @interact_manual(
        pot_shape=widgets.Dropdown(
            options=[('Zero Potential', 0),
                     ('Step Potential', 1),
                     ('Single Potential Barrier', 2),
                     ('Double Potential Barrier', 3),
                     ('Square Potential well', 4),
                     ('Harmonic Potential well', 5)],
            value=0,
            description='Shape of Potential:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        pot_height_eV=widgets.FloatSlider(
            value=78.02,min=10,max=200,step=10,
            description='Potential Height [eV]: (Height of potential wall in eV)',
            disabled=True,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        ewave=widgets.FloatSlider(
            value=80,min=10,max=300,step=10,
            description='Energy of Wavepacket [eV]:',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        dispersion_gaussian=widgets.FloatSlider(
            value=20,
            min=0.1,
            max=50.0,
            step=0.05,
            description='Gaussian Dispersion of Packet (Angstrom):',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.2f',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        ngridt=widgets.IntSlider(
            value=150,
            min=30,
            max=700,
            step=10,
            description='Number of frame (each = 0.01 fs):',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        lpacket=widgets.RadioButtons(
            options=[('wave packet', 0),
                     ('linear combination', 1),
                     ('coherent', 2)],
            value=0,
            description='packet list:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        ),
        ncombistates=widgets.IntSlider(
            value=10,
            min=1,
            max=1000,
            step=1,
            description='Number of N basis:',
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
    )
    def interactive(pot_shape, pot_height_eV, ewave, dispersion_gaussian, ngridt, lpacket, ncombistates):
        """
        Solve the time-dependent Schrödinger equation, write the wavefunction and potential data to files,
        and display the Plotly animation with Play/Pause controls.

        input:
            pot_shape           : Shape of potential (0: None, 1: Step, 2: Single, 3: Double, 4: Square, 5: Harmonic)
            pot_height_eV       : Height of the potential wall (eV)
            ewave               : Energy of the Gaussian wave packet (eV)
            dispersion_gaussian : Spatial dispersion (sigma) of the Gaussian wave packet (Angstrom)
            ngridt              : Number of time steps (each step = 0.01 fs)
            lpacket             : 0: use all basis; 1: use N basis
            ncombistates        : Number of eigenstates for linear combination (if lpacket==1)
        """
        k = (ewave * 2 / har2ev)**0.5
        sigma = dispersion_gaussian * ang2bohr
        eigval, phi = time_dep_schr_eq(pot_shape, pot_height_eV, dx)
        wave_ = make_wave(boxL_bohr, ngridx, pot_shape, sigma, k)
        write_wave(wave_, eigval, phi, boxL_bohr, ngridt, dt, ngridx, lpacket, ncombistates, au2fs_time)
        pot = potential(ngridx, pot_shape, pot_height_eV, barrier_thickness)
        write_pot(ngridx, pot)
        read_and_plot(boxL_bohr, ngridx, ngridt, ewave, lpacket, ncombistates, interval=0.05)


interactive(children=(Dropdown(description='Shape of Potential:', layout=Layout(width='400px'), options=(('Zer…